In [52]:
import numpy as np
import pandas as pd
from graphviz import Digraph

In [53]:
data = pd.read_csv('/content/drive/MyDrive/YOLO/Iris.csv')

In [54]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


**Node class**

In [55]:
class Node():
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
    '''constructor'''

    self.feature_index=feature_index
    self.threshold = threshold
    self.left =left
    self.right = right
    self.info_gain = info_gain

    self.value = value


In [56]:
#data = np.array(data)
#X = data[-1]  #Gives the last row  -> array([150, 5.9, 3.0, 5.1, 1.8, 'Iris-virginica'], dtype=object)
#X = data[:-1] #gives all rows except last one
#X = data[:]  #same as data#
#X = data[:,0:2] #contains all rows and columns 0 and 1
#X

**Tree** **class**

In [57]:
class Tree():
    def __init__(self, min_samples_split=2, max_depth=2):
        # The root of the tree
        self.root = None

        # Stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        '''recursive function to build the binary tree'''

        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)

                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])

        # Calculate leaf value
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        '''function to find the best split for the nodes'''

        # dictionary to store best split
        best_split = {}
        max_info_gain = -float("inf")

        # looping over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # calculate current Split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)

                # Checking if there are null children
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]

                    # calculate information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # Update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])

        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        '''function for information gain'''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)

        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))

        return gain

    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0

        for clas in class_labels:
            pi = len(y[y == clas]) / len(y)
            entropy += -pi * np.log2(pi)
        return entropy

    def gini_index(self, y):
        gini_index = 0
        class_labels = np.unique(y)

        for clas in class_labels:
            pi = len(y[clas == y]) / len(y)
            gini_index -= pi ** 2

        return gini_index + 1

    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)

    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
        '''function to predict single data point'''

        if tree.value is not None:
            return tree.value  # which means we have reached a leaf node
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions


In [58]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

**Fitting thr model**

In [59]:
DecisionClassifier = Tree(min_samples_split=3, max_depth=3)

DecisionClassifier.fit(X_train,y_train)

DecisionClassifier.print_tree()



X_0 <= 50.0 ? 0.34865497076023394
 left:Iris-setosa
 right:X_0 <= 100.0 ? 0.4986149584487536
  left:Iris-versicolor
  right:Iris-virginica


In [60]:
Y_pred = DecisionClassifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, Y_pred)

1.0